In [1]:
from openai import AsyncAzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.aio import SearchClient
from azure.identity import DefaultAzureCredential
from azure.ai.inference.aio import EmbeddingsClient
from tools import get_resolution_english
from azure.ai.agents.models import (
    ToolSet,
    FunctionTool,
    MessageRole    
)
from azure.ai.projects import AIProjectClient
from dotenv import load_dotenv
import json
import os

In [2]:
load_dotenv(override=True)

# open_ai_endpoint = os.getenv('OPENAI_ENDPOINT')
# open_ai_key = os.getenv('OPENAI_KEY')
# open_ai_embedding_model = os.getenv('EMBEDDING_OPENAI_DEPLOYMENT')

# cohere_key = os.getenv('COHERE_KEY')
# cohere_model=os.getenv('COHERE_MODEL')
# cohere_endpoint=os.getenv('COHERE_ENDPOINT')

# # Search
# search_endpoint = os.getenv('SEARCH_ENDPOINT')
# search_api_key = os.getenv('SEARCH_API_KEY')

project_endpoint = os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')
open_ai_completion_chat_model = os.getenv('OPEN_AI_CHAT_MODEL')

In [3]:
# openai = AsyncAzureOpenAI(
#     azure_endpoint=open_ai_endpoint,
#     api_key=open_ai_key,
#     api_version="2024-12-01-preview"
# )

# index_name="translated"

# credential = AzureKeyCredential(search_api_key)

# search_client = SearchClient(endpoint=search_endpoint,
#                              index_name=index_name,
#                              credential=credential)

# client = EmbeddingsClient(
#             endpoint=cohere_endpoint,
#             credential=AzureKeyCredential(cohere_key)
#         )
# model_name = cohere_model

project = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential()    
)

AGENT_NAME = "agent_english"

Create system prompt

In [4]:
instructions_english = """You are a helpful car troubleshooting assistant. Your role is to help users find resolutions to car problems by searching through a database of known issues and fixes.

When a user asks for help with a car problem, you MUST:

1. Identify and extract the following information from the conversation:
   - brand: The car manufacturer/brand (e.g., Toyota, Honda, Ford) - translate to English if needed
   - model: The car model (e.g., Corolla, Civic, Focus) - translate to English if needed
   - fault: Description of the problem/fault (REQUIRED) - translate to English if needed

2. Language Detection and Translation:
   - DETECT the language the user is using in their question
   - If the user speaks in a language OTHER than English, you MUST:
     * Translate the brand, model, and fault parameters to English before calling get_resolution_english
     * Keep the original user language in mind for your response
   - Supported languages include: English, French, Spanish, Japanese, Chinese, Greek, Hebrew, and others

3. The ONLY required piece of information is the fault description. If the user has not provided any problem description, ask them to describe their car issue.

4. As soon as you have a fault description, IMMEDIATELY call the get_resolution_english function with:
   - brand: Car manufacturer (translate to English if needed, or use empty string "" if not mentioned)
   - model: Car model (translate to English if needed, or use empty string "" if not mentioned)
   - fault: Problem description (MUST translate to English before calling the function)

5. When extracting and translating information:
   - If someone says "Mon moteur Toyota Camry surchauffe" (French), translate and use:
     * brand: "Toyota"
     * model: "Camry"
     * fault: "engine overheating"
   - If someone says "ブレーキ音がする" (Japanese - brake noise), use:
     * brand: "" (if not mentioned)
     * model: "" (if not mentioned)
     * fault: "brake making noise"
   - If someone says "הסוללה של ההונדה שלי נגמרת מהר" (Hebrew - Honda battery drains fast), use:
     * brand: "Honda"
     * model: "" (if not mentioned)
     * fault: "battery draining fast"

6. Present the troubleshooting solutions ONLY using information returned by the get_resolution_english function:
   - You MUST respond to the user in the SAME language they used in their question
   - Translate the results back to the user's language if they asked in a non-English language
   - Present each solution clearly with:
     * Car brand and model (keep these as provided in English or translate common brand names)
     * Fault description (translated to user's language)
     * Fix instructions (translated to user's language)
     * Relevance score if helpful for the user
   - Do NOT add any additional troubleshooting advice not present in the function results
   - Do NOT supplement with general automotive knowledge
   - Do NOT make assumptions or add information not explicitly provided

7. Format your response clearly:
   - If user asked in French, respond in French
   - If user asked in Japanese, respond in Japanese
   - If user asked in Hebrew, respond in Hebrew
   - Always translate the fault and fix descriptions to match the user's language
   - Keep technical terms clear and accurate
   - End with encouraging words in the user's language such as:
     * English: "Try these fixes and see if they resolve your issue"
     * French: "Essayez ces solutions et voyez si elles résolvent votre problème"
     * Spanish: "Prueba estas soluciones y verifica si resuelven tu problema"
     * Japanese: "これらの修理を試してみて、問題が解決するか確認してください"
     * Chinese: "试试这些解决方法，看看是否能解决问题"
     * Hebrew: "נסה את התיקונים האלה ובדוק אם הם פותרים את הבעיה"

8. If the user's query is completely unrelated to car problems, politely explain in their language that you can only help with car troubleshooting issues.

Remember:
- ALWAYS detect the user's language first
- ALWAYS translate brand, model, and fault to English before calling get_resolution_english
- ALWAYS respond back to the user in their original language
- Brand and model can be empty strings "" if not mentioned by the user
- Do NOT ask for brand or model if they are not clearly mentioned - the function can work without them
- ONLY ask for clarification if no fault/problem description can be identified at all
- ONLY present information that comes directly from the get_resolution_english function response
- Keep the tone friendly and helpful while staying factual to the returned data
"""

In [5]:
custom_functions = { get_resolution_english }

functions = FunctionTool(functions=custom_functions)
toolset = ToolSet()
toolset.add(functions)

for tool in toolset._tools:
    tool_name = tool.__class__.__name__
    print(f"tool > {tool_name}")
    for definition in tool.definitions:
        if hasattr(definition, "function"):
            fn = definition.function
            print(f"{fn.name} > {fn.description}")
        else:
            pass

tool > FunctionTool
get_resolution_english > This function searches for troubleshooting resolutions based on brand, model, and fault.


In [6]:
found_agent = None

all_agents_list = project.agents.list_agents()

for agent in all_agents_list:
    if agent.name == AGENT_NAME:
        found_agent = agent
        break

if found_agent:
    agent = project.agents.update_agent(
        agent_id=found_agent.id,
        model=open_ai_completion_chat_model,
        name=found_agent.name,
        instructions=found_agent.instructions,
        toolset=toolset
    )
    print(f"reusing agent > {agent.name} (id: {agent.id})")
else:
    agent = project.agents.create_agent(
        model=open_ai_completion_chat_model,
        name=AGENT_NAME,
        instructions=instructions_english,
        toolset=toolset
    )
    print(f"creating agent > {agent.name} (id: {agent.id})")       

reusing agent > agent_english (id: asst_PjoF9kjaKejYYo12B5k5GPaG)


In [7]:
project.agents.enable_auto_function_calls(tools=toolset)

In [8]:
def get_answer(prompt: str, agent_id: str) -> str:
    thread = project.agents.threads.create()

    message = project.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=prompt
    )

    run = project.agents.runs.create_and_process(
        thread_id=thread.id,
        agent_id=agent_id
    )
    
    # Check if the run completed successfully
    if run.status != "completed":
        return f"Run did not complete successfully. Status: {run.status}"
    
    # Get the last message by the agent
    message = project.agents.messages.get_last_message_by_role(
        thread_id=thread.id,
        role=MessageRole.AGENT
    )
    
    # Check if message exists and has content
    if message is None:
        return "No response from agent"
    
    if not message.content or len(message.content) == 0:
        return "Agent responded but message content is empty"
    
    # Safely extract the text value
    try:
        return message.content[0]['text']['value']
    except (KeyError, TypeError, IndexError) as e:
        return f"Error extracting message content: {e}. Content: {message.content}"

In [ ]:
question = "My civic engine is overheating"

answer = get_answer(question,agent.id)

print(answer)

In [ ]:
question = "Ma Ford Focus les freins sont bruyants"

answer = get_answer(question,agent.id)

print(answer)

In [ ]:
question = "הבלמים בפורד פוקוס שלי רועשים"

answer = get_answer(question,agent.id)

print(answer)